<a href="https://colab.research.google.com/github/kimyuji2/-KAIST_DSAIL-RC/blob/main/%5BWeek2%5DBPR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### BPR 노션 정리
https://www.notion.so/BPR-Bayesian-Personalized-Ranking-for-Implicit-Feedback-3f78ca28cdec4934a153145cb0d34f08

In [ ]:
import torch
import random
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Hyper parameters
lr = 0.01
reg = 0.01
latent_factors = 20

In [ ]:
from collections import defaultdict
filename = '/content/drive/MyDrive/Colab Notebooks/인턴'
train_mat = defaultdict(set)
with open(filename, 'r') as f:
  for line in f.readlines():
    u, i = line.split(" ")
    u = int(u)
    i = int(i)
    train_mat[u].add(i)

In [ ]:
num_users = 943
num_items = 1682
U = np.random.rand(num_users, latent_factors) * 0.01
V = np.random.rand(num_items, latent_factors) * 0.01
biasV = np.random.rand(num_items) * 0.01

In [ ]:
size_u_i = num_users * num_items
test_set = np.zeros(size_u_i)
test_data = np.zeros((num_users, num_items))
filename = '/content/drive/MyDrive/Colab Notebooks/인턴/test.txt'
file = open(filename, 'r')
for line in file:
  line = line.split(' ')
  user = int(line[0])
  item = int(line[1])
  test_data[user - 1][item - 1] = 1
for u in range(num_users):
    for i in range(num_items):
        if int(test_data[u][i]) == 1:
            test_set[u * num_items + i] = 1
        else:
            test_set[u * num_items + i] = 0

In [ ]:
def pre_handel(set, predict, item_count): #train_mat, predict_, num_items
    # Ensure the recommendation cannot be positive items in the training set.
    for u in set.keys():
        for j in set[u]:
            predict[(u - 1) * item_count + j - 1] = 0
    return predict

In [ ]:
# training
for i in tqdm_notebook(range(num_users)):
    for user in range(num_users):
        # sample a user
        u = random.randint(1, num_users)
        if u not in train_mat.keys():
            continue
        # sample a positive item from the observed items
        i = random.sample(train_mat[u], 1)[0]
        # sample a negative item from the unobserved items
        j = random.randint(1, num_items)
        while j in train_mat[u]:
            j = random.randint(1, num_items)
        u -= 1
        i -= 1
        j -= 1
        r_ui = np.dot(U[u], V[i].T) + biasV[i]
        r_uj = np.dot(U[u], V[j].T) + biasV[j]
        r_uij = r_ui - r_uj
        loss_func = -1.0 / (1 + np.exp(r_uij)) # sigmoid
        # update U and V
        U[u] += -lr * (loss_func * (V[i] - V[j]) + reg * U[u])
        V[i] += -lr * (loss_func * U[u] + reg * V[i])
        V[j] += -lr * (loss_func * (-U[u]) + reg * V[j])
        # update biasV
        biasV[i] += -lr * (loss_func + reg * biasV[i])
        biasV[j] += -lr * (-loss_func + reg * biasV[j])
predict_matrix = np.mat(U) * np.mat(V.T)
# prediction
predict_ = predict_matrix.getA().reshape(-1)
predict_ = pre_handel(train_mat, predict_, num_items)
auc_score = roc_auc_score(test_set, predict_)
print('AUC:', auc_score)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



AUC: 0.8159500589161064
